In [72]:
import pandas as pd
import requests

pd.set_option('display.max_columns', None)

In [206]:
date = "2017-JUL-31"

#game slates and salaries

response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/DfsSlatesByDate/{date}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
games = response.json()

#player points

response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/PlayerGameStatsByDate/{date}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
players2 = response.json()

In [232]:
#format json

#game slates
df = pd.json_normalize(games)

#flatten
df_games = pd.json_normalize(games, record_path =['DfsSlateGames'])
df_player_sal = pd.json_normalize(games, record_path =['DfsSlatePlayers'])

#player points
player_points = pd.json_normalize(players2)

In [233]:
points_and_sal = df_player_sal.merge(player_points,on='PlayerID')

points_and_sal = points_and_sal[points_and_sal['SlateID']==1977]

In [234]:
points_and_sal = points_and_sal.loc[:,['OperatorPlayerName', 'OperatorPosition', 'OperatorSalary','FantasyPoints']]

points_and_sal['OperatorPlayerName'] = points_and_sal['OperatorPlayerName'].str.replace(' ','')

points_and_sal['projected'] = points_and_sal['OperatorSalary']/1000

#points_and_sal['Position'] = points_and_sal['OperatorPosition'].str.split('/')

#points_and_sal = points_and_sal.explode('Position')

points_and_sal.head(20)

,OperatorPlayerName,OperatorPosition,OperatorSalary,FantasyPoints,projected
0,LuisSeverino,SP,11900,8.00,11.9
8,GioGonzález,SP,11200,12.00,11.2
16,CharlieMorton,SP,10600,8.00,10.6
25,MichaelFulmer,SP,8100,0.00,8.1
33,MikeClevinger,SP,7800,-2.00,7.8
41,DannyDuffy,SP,8900,6.00,8.9
49,FelixHernandez,SP,8500,1.33,8.5
58,AlexCobb,SP,7300,-5.00,7.3
67,MikeFoltynewicz,SP,9200,-1.00,9.2
70,MarcoEstrada,SP,6500,6.00,6.5


In [251]:
from pulp import *
import numpy as np


def generate_optimal_lineup(players, positions, values, costs, budget):
    num_variables = len(players)
    
    lp = LpProblem("My LP Problem", pulp.LpMaximize)
    
    d = {}
    for i in range(0, num_variables):
        d[players[i]] = LpVariable(players[i], cat="Binary")
    
    objective = sum(np.array(values) * np.array(list(d.values())))
    lp += objective
    
    p_constraint = 0
    c_constraint = 0
    oneb_constraint = 0
    twob_constraint = 0
    threeb_constraint = 0
    ss_constraint = 0
    of_constraint = 0
    for i in range(0, len(positions)):
        if positions[i] == "SP":
            p_constraint += d[players[i]]
        elif positions[i] == "C":
            c_constraint += d[players[i]]
        elif positions[i] == "1B":
            oneb_constraint += d[players[i]]
        elif positions[i] == "2B":
            twob_constraint += d[players[i]]
        elif positions[i] == "3B":
            threeb_constraint += d[players[i]]
        elif positions[i] == "SS":
            ss_constraint += d[players[i]]
        else:
            of_constraint += d[players[i]]
    lp += p_constraint == 2
    lp += c_constraint == 1
    lp += oneb_constraint == 1
    lp += twob_constraint == 1
    lp += threeb_constraint == 1
    lp += ss_constraint == 1
    lp += of_constraint == 3
    
    cost = sum(np.array(costs) * np.array(list(d.values())))
    lp += cost <= budget
    
    lp.solve()
    
    lineup = [variable.name for variable in lp.variables() if variable.varValue == 1]
    
    print(lineup)
    return lineup
    #print(sum(np.array(costs)))
    #return lp

In [264]:
budget = 50000

players = points_and_sal['OperatorPlayerName'].tolist()
positions = points_and_sal['OperatorPosition'].tolist()
costs = points_and_sal['OperatorSalary'].tolist()
values = points_and_sal['FantasyPoints'].tolist()


x = generate_optimal_lineup(players, positions, values, costs, budget)

['AlexBregman', 'CoreyDickerson', 'CésarHernández', 'DerekFisher', 'DougFister', 'GioGonzález', 'JakeMarisnick', 'KurtSuzuki', 'MarcusSemien', 'TommyJoseph']


/Users/frankbruni/opt/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [267]:
lineup = points_and_sal[points_and_sal['OperatorPlayerName'].isin(x)]

lineup

,OperatorPlayerName,OperatorPosition,OperatorSalary,FantasyPoints,projected
8,GioGonzález,SP,11200,12.00,11.2
199,DerekFisher,OF,3700,11.00,3.7
299,CoreyDickerson,OF,4200,14.00,4.2
426,MarcusSemien,SS,4300,11.00,4.3
443,CésarHernández,2B,3800,9.00,3.8
630,KurtSuzuki,C,3600,7.00,3.6
751,AlexBregman,3B,3800,11.00,3.8
791,TommyJoseph,1B,3600,6.00,3.6
1182,JakeMarisnick,OF,3400,20.00,3.4
1384,DougFister,SP,4500,9.66,4.5


In [268]:
lineup['OperatorSalary'].sum()

46100